<a href="https://colab.research.google.com/github/vasiliyeskin/MachineLearningExperiences/blob/master/FashionMNIST/fashion_mnist_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оптимизация гиперпараметров нейросети с помощью [Keras Tuner](https://github.com/keras-team/keras-tuner)

Взято с https://www.youtube.com/watch?v=M08yHFPKTNI&list=PLtPJ9lKvJ4oiz9aaL_xcZd-x0qd8G0VN_&index=11.

Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

Не забудьте подключить GPU, чтобы сеть обучалась быстрее (Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU).



## Гиперпараметры обучения нейронной сети

- Количество слоев нейронной сети
- Количество нейронов в каждом слое
- Функции активации, которые используются в слоях
- Тип оптимизатора при обучении нейронной сети
- Количество эпох обучения

## Установка Keras Tuner

In [23]:
pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


## Подключаем нужные пакеты

In [31]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from google.colab import files
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

## Подготовка данных для обучения сети

In [32]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [33]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255 
x_test = x_test / 255 
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

## Задаем функцию создания нейронной сети

In [34]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 128
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation=activation_choice))   
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Создаем tuner

Доступные типы тюнеров: 
- RandomSearch - случайный поиск.
- Hyperband - алгоритм оптимизации на основе многорукого бандита, Li, Lisha, and Kevin Jamieson. ["Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization."Journal of Machine Learning Research 18 (2018): 1-52](http://jmlr.org/papers/v18/16-558.html).
- BayesianOptimization - [байесовская оптимизация](https://en.wikipedia.org/wiki/Bayesian_optimization).

In [35]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=80,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )

INFO:tensorflow:Reloading Oracle from existing project test_directory/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from test_directory/untitled_project/tuner0.json


## Запускаем подбор гиперпараметров

Пространство поиска

In [36]:
tuner.search_space_summary()

Подбор гиперпараметров

In [37]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=20,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Epoch 1/20
188/188 [==============================] - 2s 9ms/step - loss: 0.6341 - accuracy: 0.7696 - val_loss: 0.5599 - val_accuracy: 0.7902
Epoch 2/20
188/188 [==============================] - 1s 7ms/step - loss: 0.4195 - accuracy: 0.8424 - val_loss: 0.4258 - val_accuracy: 0.8422
Epoch 3/20
188/188 [==============================] - 2s 8ms/step - loss: 0.3632 - accuracy: 0.8650 - val_loss: 0.4097 - val_accuracy: 0.8518
Epoch 4/20
188/188 [==============================] - 1s 8ms/step - loss: 0.3298 - accuracy: 0.8775 - val_loss: 0.3688 - val_accuracy: 0.8712
Epoch 5/20
188/188 [==============================] - 2s 8ms/step - loss: 0.3031 - accuracy: 0.8854 - val_loss: 0.3859 - val_accuracy: 0.8704
Epoch 6/20
188/188 [==============================] - 1s 8ms/step - loss: 0.2846 - accuracy: 0.8926 - val_loss: 0.4103 - val_accuracy: 0.8613
Epoch 7/20
188/188 [==============================] - 1s 8ms/step - loss: 0.2711 - accuracy: 0.8979 - val_loss: 0.3620 - val_accuracy: 0.8733
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 8ms/step - loss: 0.5366 - accuracy: 0.8092 - val_loss: 0.4203 - val_accuracy: 0.8470
Epoch 2/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3972 - accuracy: 0.8564 - val_loss: 0.3902 - val_accuracy: 0.8573
Epoch 3/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3593 - accuracy: 0.8692 - val_loss: 0.3583 - val_accuracy: 0.8701
Epoch 4/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3280 - accuracy: 0.8807 - val_loss: 0.3493 - val_accuracy: 0.8736
Epoch 5/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3068 - accuracy: 0.8861 - val_loss: 0.3395 - val_accuracy: 0.8776
Epoch 6/20
188/188 [==============================] - 1s 7ms/step - loss: 0.2964 - accuracy: 0.8898 - val_loss: 0.3457 - val_accuracy: 0.8749
Epoch 7/20
188/188 [==============================] - 1s 7ms/step - loss: 0.2807 - accuracy: 0.8950 - val_loss: 0.3441 - val_accuracy: 0.8780
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 7ms/step - loss: 1.0759 - accuracy: 0.6749 - val_loss: 0.7606 - val_accuracy: 0.7538
Epoch 2/20
188/188 [==============================] - 1s 7ms/step - loss: 0.6991 - accuracy: 0.7733 - val_loss: 0.6408 - val_accuracy: 0.7854
Epoch 3/20
188/188 [==============================] - 1s 7ms/step - loss: 0.6140 - accuracy: 0.7977 - val_loss: 0.5854 - val_accuracy: 0.8023
Epoch 4/20
188/188 [==============================] - 1s 7ms/step - loss: 0.5673 - accuracy: 0.8105 - val_loss: 0.5532 - val_accuracy: 0.8099
Epoch 5/20
188/188 [==============================] - 1s 7ms/step - loss: 0.5373 - accuracy: 0.8190 - val_loss: 0.5276 - val_accuracy: 0.8161
Epoch 6/20
188/188 [==============================] - 1s 7ms/step - loss: 0.5161 - accuracy: 0.8245 - val_loss: 0.5087 - val_accuracy: 0.8213
Epoch 7/20
188/188 [==============================] - 1s 7ms/step - loss: 0.4996 - accuracy: 0.8289 - val_loss: 0.5016 - val_accuracy: 0.8206
Epoch 

Epoch 1/20
188/188 [==============================] - 2s 8ms/step - loss: 0.6241 - accuracy: 0.7946 - val_loss: 0.4417 - val_accuracy: 0.8422
Epoch 2/20
188/188 [==============================] - 1s 8ms/step - loss: 0.4071 - accuracy: 0.8538 - val_loss: 0.4196 - val_accuracy: 0.8485
Epoch 3/20
188/188 [==============================] - 1s 8ms/step - loss: 0.3649 - accuracy: 0.8671 - val_loss: 0.3719 - val_accuracy: 0.8675
Epoch 4/20
188/188 [==============================] - 2s 8ms/step - loss: 0.3311 - accuracy: 0.8787 - val_loss: 0.3625 - val_accuracy: 0.8703
Epoch 5/20
188/188 [==============================] - 1s 8ms/step - loss: 0.3217 - accuracy: 0.8826 - val_loss: 0.3748 - val_accuracy: 0.8647
Epoch 6/20
188/188 [==============================] - 1s 8ms/step - loss: 0.3036 - accuracy: 0.8879 - val_loss: 0.3415 - val_accuracy: 0.8747
Epoch 7/20
188/188 [==============================] - 1s 7ms/step - loss: 0.2823 - accuracy: 0.8950 - val_loss: 0.3193 - val_accuracy: 0.8849
Epoch 

Epoch 1/20
188/188 [==============================] - 2s 9ms/step - loss: 0.7911 - accuracy: 0.7299 - val_loss: 0.4542 - val_accuracy: 0.8338
Epoch 2/20
188/188 [==============================] - 2s 8ms/step - loss: 0.4725 - accuracy: 0.8235 - val_loss: 0.4152 - val_accuracy: 0.8442
Epoch 3/20
188/188 [==============================] - 2s 8ms/step - loss: 0.4021 - accuracy: 0.8490 - val_loss: 0.4390 - val_accuracy: 0.8361
Epoch 4/20
188/188 [==============================] - 2s 8ms/step - loss: 0.3653 - accuracy: 0.8629 - val_loss: 0.3786 - val_accuracy: 0.8572
Epoch 5/20
188/188 [==============================] - 2s 8ms/step - loss: 0.3362 - accuracy: 0.8731 - val_loss: 0.3307 - val_accuracy: 0.8802
Epoch 6/20
188/188 [==============================] - 2s 8ms/step - loss: 0.3157 - accuracy: 0.8804 - val_loss: 0.3539 - val_accuracy: 0.8695
Epoch 7/20
188/188 [==============================] - 2s 8ms/step - loss: 0.2947 - accuracy: 0.8888 - val_loss: 0.3310 - val_accuracy: 0.8777
Epoch 

...

Epoch 1/20
188/188 [==============================] - 2s 8ms/step - loss: 0.7909 - accuracy: 0.7295 - val_loss: 0.5253 - val_accuracy: 0.8148
Epoch 2/20
188/188 [==============================] - 2s 8ms/step - loss: 0.4829 - accuracy: 0.8181 - val_loss: 0.4447 - val_accuracy: 0.8234
Epoch 3/20
188/188 [==============================] - 2s 8ms/step - loss: 0.4060 - accuracy: 0.8479 - val_loss: 0.3940 - val_accuracy: 0.8520
Epoch 4/20
188/188 [==============================] - 2s 8ms/step - loss: 0.3642 - accuracy: 0.8621 - val_loss: 0.3776 - val_accuracy: 0.8607
Epoch 5/20
188/188 [==============================] - 2s 8ms/step - loss: 0.3372 - accuracy: 0.8739 - val_loss: 0.3663 - val_accuracy: 0.8692
Epoch 6/20
188/188 [==============================] - 1s 8ms/step - loss: 0.3103 - accuracy: 0.8838 - val_loss: 0.3263 - val_accuracy: 0.8788
Epoch 7/20
188/188 [==============================] - 2s 8ms/step - loss: 0.2929 - accuracy: 0.8896 - val_loss: 0.3535 - val_accuracy: 0.8692
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 8ms/step - loss: 0.6382 - accuracy: 0.7939 - val_loss: 0.4297 - val_accuracy: 0.8478
Epoch 2/20
188/188 [==============================] - 1s 7ms/step - loss: 0.4139 - accuracy: 0.8506 - val_loss: 0.4175 - val_accuracy: 0.8524
Epoch 3/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3708 - accuracy: 0.8650 - val_loss: 0.4068 - val_accuracy: 0.8478
Epoch 4/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3442 - accuracy: 0.8731 - val_loss: 0.3539 - val_accuracy: 0.8736
Epoch 5/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3213 - accuracy: 0.8823 - val_loss: 0.3581 - val_accuracy: 0.8705
Epoch 6/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3040 - accuracy: 0.8887 - val_loss: 0.3696 - val_accuracy: 0.8637
Epoch 7/20
188/188 [==============================] - 1s 8ms/step - loss: 0.2883 - accuracy: 0.8938 - val_loss: 0.3374 - val_accuracy: 0.8799
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 7ms/step - loss: 0.7891 - accuracy: 0.7367 - val_loss: 0.5360 - val_accuracy: 0.7990
Epoch 2/20
188/188 [==============================] - 1s 7ms/step - loss: 0.4630 - accuracy: 0.8345 - val_loss: 0.4247 - val_accuracy: 0.8449
Epoch 3/20
188/188 [==============================] - 1s 7ms/step - loss: 0.4078 - accuracy: 0.8544 - val_loss: 0.4060 - val_accuracy: 0.8503
Epoch 4/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3850 - accuracy: 0.8604 - val_loss: 0.3969 - val_accuracy: 0.8537
Epoch 5/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3641 - accuracy: 0.8688 - val_loss: 0.3797 - val_accuracy: 0.8612
Epoch 6/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3464 - accuracy: 0.8743 - val_loss: 0.3659 - val_accuracy: 0.8649
Epoch 7/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3341 - accuracy: 0.8793 - val_loss: 0.3533 - val_accuracy: 0.8696
Epoch 

Epoch 1/20
188/188 [==============================] - 1s 8ms/step - loss: 0.5176 - accuracy: 0.8151 - val_loss: 0.3963 - val_accuracy: 0.8558
Epoch 2/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3890 - accuracy: 0.8581 - val_loss: 0.3805 - val_accuracy: 0.8626
Epoch 3/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3560 - accuracy: 0.8686 - val_loss: 0.3794 - val_accuracy: 0.8640
Epoch 4/20
188/188 [==============================] - 1s 7ms/step - loss: 0.3202 - accuracy: 0.8823 - val_loss: 0.3601 - val_accuracy: 0.8695
Epoch 5/20
188/188 [==============================] - 1s 8ms/step - loss: 0.3022 - accuracy: 0.8876 - val_loss: 0.3364 - val_accuracy: 0.8758
Epoch 6/20
188/188 [==============================] - 1s 8ms/step - loss: 0.2849 - accuracy: 0.8939 - val_loss: 0.3344 - val_accuracy: 0.8785
Epoch 7/20
188/188 [==============================] - 1s 8ms/step - loss: 0.2689 - accuracy: 0.9012 - val_loss: 0.3152 - val_accuracy: 0.8863
Epoch 

INFO:tensorflow:Oracle triggered exit


## Выбираем лучшую модель

In [38]:
tuner.results_summary()

Получаем три лучших модели

In [39]:
models = tuner.get_best_models(num_models=8)

Оцениваем качество модели на тестовых данных

In [40]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 736)               577760    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               188672    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 769,002
Trainable params: 769,002
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 1s 4ms/step - loss: 0.3647 - accuracy: 0.8909

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 672)               527520    
_________________________________________________________________
dense_